In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


# Read Raw Data

In [2]:
import os 
import pandas as pd

RecName = 'Diag'
folder = 'data/Simulations/'
raw_file_path =  os.path.join(folder, RecName) + '.csv'
print(raw_file_path)


data/Simulations/Diag.csv


In [3]:

# load all records
df = pd.read_csv(raw_file_path)
df

,PID,ECID,RID,DT,Value
0,P0,EC140,161119,2023/3/19,N91.5
1,P0,EC23,214372,2023/3/19,S53.0
2,P0,EC141,187825,2023/3/20,P24.5
3,P0,EC208,227678,2023/3/20,S21.5
4,P0,EC19,174522,2023/3/20,U59.5
...,...,...,...,...,...
921,P6,EC15,240970,2023/4/7,M43.5
922,P7,EC17,209056,2023/3/18,S37.5
923,P7,EC17,209055,2023/3/18,J24.5
924,P7,EC17,248716,2023/3/24,J74.5


# Preprocess

## Add Missing Records

In [4]:
from recfldgrn.datapoint import load_df_data_from_folder

prefix_cols = ['PID', 'ECID']
focal_cols = ['RID']

Parent_RecName = 'EC'
rec_folder = 'data/ProcData/RecFolder/'
fullrec_folder =  os.path.join(rec_folder, Parent_RecName) 
df_parent = load_df_data_from_folder(fullrec_folder)
df_parent

,PID,ECID,DT_min,DT_max,BasicInfo
0,P0,EC18,2023-03-18,2023-04-16,D&X
1,P0,EC140,2023-03-19,2023-04-05,D&X
2,P0,EC23,2023-03-19,2023-04-16,C&W
3,P0,EC141,2023-03-20,2023-04-16,C&Z
4,P0,EC142,2023-03-20,2023-04-15,B&W
...,...,...,...,...,...
220,P6,EC15,2023-03-21,2023-04-09,B&X
221,P6,EC93,2023-03-23,2023-04-04,B&X
222,P6,EC195,2023-03-25,2023-03-25,C&Z
223,P6,EC173,2023-04-05,2023-04-13,C&W


In [5]:
import numpy as np

d = pd.merge(df_parent[prefix_cols], df, on = prefix_cols, how = 'left')
for col in focal_cols: 
    d.loc[d[col].isnull(), col] = np.ones(d[col].isnull().sum()).cumsum() + d[col].max()
    d[col] = d[col].astype(int)
    assert len(d[col]) == d[col].nunique()
    
df = d
df

,PID,ECID,RID,DT,Value
0,P0,EC18,200972,2023/4/10,D14.0
1,P0,EC18,227676,2023/4/13,J19.5
2,P0,EC18,214361,2023/4/16,J74.5
3,P0,EC18,200975,2023/4/8,I94.0
4,P0,EC140,161119,2023/3/19,N91.5
...,...,...,...,...,...
929,P6,EC173,174622,2023/4/5,R80.5
930,P7,EC17,209056,2023/3/18,S37.5
931,P7,EC17,209055,2023/3/18,J24.5
932,P7,EC17,248716,2023/3/24,J74.5


## Update DT Format

In [6]:
df['DT'] = pd.to_datetime(df['DT'])

# Save To Disk by Bucket

In [7]:
from recfldgrn.datapoint import convert_PID_to_PIDgroup
from recfldgrn.datapoint import RANGE_SIZE, write_df_to_folders, load_df_data_from_folder

In [8]:
rec_folder = 'data/ProcData/RecFolder'
fullrec_folder = os.path.join(rec_folder, RecName)
print(df.shape, fullrec_folder)

if not os.path.exists(fullrec_folder): os.makedirs(fullrec_folder)
write_df_to_folders(RecName, fullrec_folder, df)


(934, 5) data/ProcData/RecFolder/Diag


In [9]:
from recfldgrn.datapoint import PatientDP
print(rec_folder)
PID = 'P4'
Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
print(Pat)
dfx = Pat.get_df_rec(RecName)
dfx

data/ProcData/RecFolder


,PID,ECID,RID,DT,Value
0,P4,EC125,238249,2023-03-18,N97.5
1,P4,EC125,158555,2023-03-22,X4.5
2,P4,EC125,158547,2023-04-02,P36.5
3,P4,EC24,238252,2023-03-18,C91.0
4,P4,EC24,145433,2023-03-22,H48.5
...,...,...,...,...,...
79,P4,EC121,158548,2023-04-16,R28.5
80,P4,EC196,248719,NaT,NaN
81,P4,EC203,238255,2023-04-04,P40.5
82,P4,EC204,145439,2023-04-12,U55.0


In [10]:
from recfldgrn.datapoint import load_df_data_from_folder
print(fullrec_folder)
df = load_df_data_from_folder(fullrec_folder)
df

data/ProcData/RecFolder/Diag


,PID,ECID,RID,DT,Value
0,P0,EC18,200972,2023-04-10,D14.0
1,P0,EC18,227676,2023-04-13,J19.5
2,P0,EC18,214361,2023-04-16,J74.5
3,P0,EC18,200975,2023-04-08,I94.0
4,P0,EC140,161119,2023-03-19,N91.5
...,...,...,...,...,...
929,P6,EC173,174622,2023-04-05,R80.5
930,P7,EC17,209056,2023-03-18,S37.5
931,P7,EC17,209055,2023-03-18,J24.5
932,P7,EC17,248716,2023-03-24,J74.5


# Field & Grain

In [11]:
FieldGrainInfo = []

In [12]:
RecName = 'Diag'
prefix_ids = ['PID', 'ECID']
focal_ids = ['RID']
field_list = ['Value', 'DT']

## Field `Value`

In [13]:
# (a) field and (b) get field-focused df
field = 'Value'
dfx = df[prefix_ids + focal_ids + [field]]
dfx

,PID,ECID,RID,Value
0,P0,EC18,200972,D14.0
1,P0,EC18,227676,J19.5
2,P0,EC18,214361,J74.5
3,P0,EC18,200975,I94.0
4,P0,EC140,161119,N91.5
...,...,...,...,...
929,P6,EC173,174622,R80.5
930,P7,EC17,209056,S37.5
931,P7,EC17,209055,J24.5
932,P7,EC17,248716,J74.5


### Grain `Diag@Value-DiagDftGrn`

In [14]:
from recfldgrn.grains import Field2Grain_2_get_GrnStrList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnIdxList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnWgtList_Fn
from recfldgrn.utils import generate_grain_vocab_info

In [15]:
# (c) grain
grain = 'DiagDftGrn'
field2grain = '-'.join([field, grain])
recfield2grain = RecName + '@' + field2grain
print(grain)
print(field2grain)
print(recfield2grain)

DiagDftGrn
Value-DiagDftGrn
Diag@Value-DiagDftGrn


In [16]:
# grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
# s = dfx[field].apply(lambda x: grain_str_fn(x))
# s

In [17]:
# Vocab = generate_grain_vocab_info(s) # load from disk
# v2idx = Vocab['v2idx']
# v2idx

In [18]:
# grain_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
# s = dfx[field].apply(lambda x: grain_idx_fn(x, v2idx))
# s

In [19]:
from recfldgrn.grains.misc.dt_grn import load_DTgrn_Dictionary
from transformers import AutoTokenizer

if 'Nume' in grain:
    # print('Current grain is a numeric grain')
    grain_wgt_fn = Field2Grain_2_get_GrnWgtList_Fn[recfield2grain]
    series_wgt = dfx[field].apply(grain_wgt_fn)
    Vocab = {'v2idx': len(series_wgt.iloc[0]) * [1]}
    v2idx = Vocab['v2idx']

elif 'DT' in grain:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = load_DTgrn_Dictionary()
    v2idx = Vocab['v2idx']
    
elif 'Tknz' in grain:
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    v2idx = tokenizer
    Vocab = {'v2idx': v2idx}

else:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = generate_grain_vocab_info(series_str)
    v2idx = Vocab['v2idx']

print(v2idx)
grain_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
series_idx = dfx[field].apply(lambda x: grain_idx_fn(x, v2idx))
series_idx


{'_missing': 0, 'C-5': 1, 'C-0': 2, 'A-R': 3, 'A-U': 4, 'A-Q': 5, 'A-X': 6, 'A-B': 7, 'A-J': 8, 'A-P': 9, 'A-G': 10, 'A-Y': 11, 'A-D': 12, 'A-M': 13, 'A-H': 14, 'A-N': 15, 'A-S': 16, 'A-O': 17, 'A-F': 18, 'A-E': 19, 'A-L': 20, 'A-W': 21, 'A-V': 22, 'A-K': 23, 'A-T': 24, 'A-C': 25, 'A-Z': 26, 'A-A': 27, 'A-I': 28, 'B-M44': 29, 'B-Y41': 30, 'B-G47': 31, 'B-B42': 32, 'B-U55': 33, 'B-H75': 34, 'B-D96': 35, 'B-J74': 36, 'B-O5.': 37, 'B-U84': 38, 'B-Q32': 39, 'B-D33': 40, 'B-G57': 41, 'B-N34': 42, 'B-X4.': 43, 'B-B82': 44, 'B-G91': 45, 'B-J24': 46, 'B-R36': 47, 'B-A94': 48, 'B-G71': 49, 'B-I47': 50, 'B-R37': 51, 'B-N22': 52, 'B-R15': 53, 'B-R12': 54, 'B-X89': 55, 'B-S82': 56, 'B-K25': 57, 'B-W80': 58, 'B-R10': 59, 'B-E71': 60, 'B-J86': 61, 'B-U68': 62, 'B-L91': 63, 'B-P40': 64, 'B-N78': 65, 'B-P48': 66, 'B-K28': 67, 'B-E2.': 68, 'B-B12': 69, 'B-E53': 70, 'B-G2.': 71, 'B-T90': 72, 'B-C30': 73, 'B-P97': 74, 'B-S63': 75, 'B-X86': 76, 'B-U23': 77, 'B-Y38': 78, 'B-P23': 79, 'B-X85': 80, 'B-D8.': 

0      [12, 656, 2]
1       [8, 578, 1]
2        [8, 36, 1]
3      [28, 579, 2]
4      [15, 580, 1]
           ...     
929      [3, 84, 1]
930    [16, 799, 1]
931      [8, 46, 1]
932      [8, 36, 1]
933     [13, 29, 1]
Name: Value, Length: 934, dtype: object

In [20]:
series_idx

0      [12, 656, 2]
1       [8, 578, 1]
2        [8, 36, 1]
3      [28, 579, 2]
4      [15, 580, 1]
           ...     
929      [3, 84, 1]
930    [16, 799, 1]
931      [8, 46, 1]
932      [8, 36, 1]
933     [13, 29, 1]
Name: Value, Length: 934, dtype: object

In [21]:
d = {}
d['RecName'] = RecName

d['prefix_ids'] = prefix_ids
d['focal_ids'] = focal_ids 
d['field_list'] = field_list

d['field'] = field
d['grain'] = grain
d['field2grain'] = field2grain
d['recfield2grain'] = recfield2grain
d['Vocab'] = Vocab

FieldGrainInfo.append(d)

In [22]:
d

{'RecName': 'Diag',
 'prefix_ids': ['PID', 'ECID'],
 'focal_ids': ['RID'],
 'field_list': ['Value', 'DT'],
 'field': 'Value',
 'grain': 'DiagDftGrn',
 'field2grain': 'Value-DiagDftGrn',
 'recfield2grain': 'Diag@Value-DiagDftGrn',
 'Vocab': {'idx2v': {0: '_missing',
   1: 'C-5',
   2: 'C-0',
   3: 'A-R',
   4: 'A-U',
   5: 'A-Q',
   6: 'A-X',
   7: 'A-B',
   8: 'A-J',
   9: 'A-P',
   10: 'A-G',
   11: 'A-Y',
   12: 'A-D',
   13: 'A-M',
   14: 'A-H',
   15: 'A-N',
   16: 'A-S',
   17: 'A-O',
   18: 'A-F',
   19: 'A-E',
   20: 'A-L',
   21: 'A-W',
   22: 'A-V',
   23: 'A-K',
   24: 'A-T',
   25: 'A-C',
   26: 'A-Z',
   27: 'A-A',
   28: 'A-I',
   29: 'B-M44',
   30: 'B-Y41',
   31: 'B-G47',
   32: 'B-B42',
   33: 'B-U55',
   34: 'B-H75',
   35: 'B-D96',
   36: 'B-J74',
   37: 'B-O5.',
   38: 'B-U84',
   39: 'B-Q32',
   40: 'B-D33',
   41: 'B-G57',
   42: 'B-N34',
   43: 'B-X4.',
   44: 'B-B82',
   45: 'B-G91',
   46: 'B-J24',
   47: 'B-R36',
   48: 'B-A94',
   49: 'B-G71',
   50: 'B-I47',

## Field `DT`

In [23]:
# (a) field and (b) get field-focused df
field = 'DT'
dfx = df[prefix_ids + focal_ids + [field]]
dfx

,PID,ECID,RID,DT
0,P0,EC18,200972,2023-04-10
1,P0,EC18,227676,2023-04-13
2,P0,EC18,214361,2023-04-16
3,P0,EC18,200975,2023-04-08
4,P0,EC140,161119,2023-03-19
...,...,...,...,...
929,P6,EC173,174622,2023-04-05
930,P7,EC17,209056,2023-03-18
931,P7,EC17,209055,2023-03-18
932,P7,EC17,248716,2023-03-24


### DTDftGrn

In [24]:
# (c) grain
grain = 'DTDftGrn'
field2grain = '-'.join([field, grain])
recfield2grain = RecName + '@' + field2grain
print(grain)
print(field2grain)
print(recfield2grain)

DTDftGrn
DT-DTDftGrn
Diag@DT-DTDftGrn


In [25]:
from recfldgrn.grains.misc.dt_grn import load_DTgrn_Dictionary
from transformers import AutoTokenizer

if 'Nume' in grain:
    # print('Current grain is a numeric grain')
    grain_wgt_fn = Field2Grain_2_get_GrnWgtList_Fn[recfield2grain]
    series_wgt = dfx[field].apply(grain_wgt_fn)
    Vocab = {'v2idx': len(series_wgt.iloc[0]) * [1]}
    v2idx = Vocab['v2idx']

elif 'DT' in grain:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = load_DTgrn_Dictionary()
    v2idx = Vocab['v2idx']
    
elif 'Tknz' in grain:
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    v2idx = tokenizer
    Vocab = {'v2idx': v2idx}

else:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = generate_grain_vocab_info(series_str)
    v2idx = Vocab['v2idx']

print(v2idx)
grain_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
series_idx = dfx[field].apply(lambda x: grain_idx_fn(x, v2idx))
series_idx


{'_missing': 0, 'Yx': 1, 'Y2010': 2, 'Y2011': 3, 'Y2012': 4, 'Y2013': 5, 'Y2014': 6, 'Y2015': 7, 'Y2016': 8, 'Y2017': 9, 'Y2018': 10, 'Y2019': 11, 'Y2020': 12, 'Y2021': 13, 'Y2022': 14, 'Y2023': 15, 'Y2024': 16, 'Y2025': 17, 'Y2026': 18, 'Y2027': 19, 'Y2028': 20, 'Y2029': 21, 'Mx': 22, 'M1': 23, 'M2': 24, 'M3': 25, 'M4': 26, 'M5': 27, 'M6': 28, 'M7': 29, 'M8': 30, 'M9': 31, 'M10': 32, 'M11': 33, 'M12': 34, 'Dx': 35, 'D1': 36, 'D2': 37, 'D3': 38, 'D4': 39, 'D5': 40, 'D6': 41, 'D7': 42, 'D8': 43, 'D9': 44, 'D10': 45, 'D11': 46, 'D12': 47, 'D13': 48, 'D14': 49, 'D15': 50, 'D16': 51, 'D17': 52, 'D18': 53, 'D19': 54, 'D20': 55, 'D21': 56, 'D22': 57, 'D23': 58, 'D24': 59, 'D25': 60, 'D26': 61, 'D27': 62, 'D28': 63, 'D29': 64, 'D30': 65, 'D31': 66, 'Sx': 67, 'S1': 68, 'S2': 69, 'S3': 70, 'S4': 71, 'Wx': 72, 'W1': 73, 'W2': 74, 'W3': 75, 'W4': 76, 'W5': 77, 'W6': 78, 'W7': 79, 'WYx': 80, 'WY1': 81, 'WY2': 82, 'WY3': 83, 'WY4': 84, 'WY5': 85, 'WY6': 86, 'WY7': 87, 'WY8': 88, 'WY9': 89, 'WY10': 

0      [15, 26, 45, 69, 73, 96, 137]
1      [15, 26, 48, 69, 76, 96, 137]
2      [15, 26, 51, 69, 79, 96, 137]
3      [15, 26, 43, 69, 78, 95, 137]
4      [15, 25, 54, 68, 79, 92, 137]
                   ...              
929    [15, 26, 40, 69, 75, 95, 137]
930    [15, 25, 53, 68, 78, 92, 137]
931    [15, 25, 53, 68, 78, 92, 137]
932    [15, 25, 59, 68, 77, 93, 137]
933    [15, 25, 62, 68, 73, 94, 137]
Name: DT, Length: 934, dtype: object

In [26]:
d = {}
d['RecName'] = RecName

d['prefix_ids'] = prefix_ids
d['focal_ids'] = focal_ids 
d['field_list'] = field_list


d['field'] = field
d['grain'] = grain
d['field2grain'] = field2grain
d['recfield2grain'] = recfield2grain
d['Vocab'] = Vocab

FieldGrainInfo.append(d)

# Save Field Grain Info



In [27]:
df_FieldGrainInfo = pd.DataFrame(FieldGrainInfo)
df_FieldGrainInfo

,RecName,prefix_ids,focal_ids,field_list,field,grain,field2grain,recfield2grain,Vocab
0,Diag,"[PID, ECID]",[RID],"[Value, DT]",Value,DiagDftGrn,Value-DiagDftGrn,Diag@Value-DiagDftGrn,"{'idx2v': {0: '_missing', 1: 'C-5', 2: 'C-0', ..."
1,Diag,"[PID, ECID]",[RID],"[Value, DT]",DT,DTDftGrn,DT-DTDftGrn,Diag@DT-DTDftGrn,"{'idx2v': {0: '_missing', 1: 'Yx', 2: 'Y2010',..."


In [28]:
fldgrn_folder = 'data/ProcData/FldGrnInfo'

if not os.path.exists(fldgrn_folder): os.makedirs(fldgrn_folder)

fullfldgrn_file = os.path.join(fldgrn_folder, RecName + '.p')
df_FieldGrainInfo.to_pickle(fullfldgrn_file)